#4.5 학습 알고리즘 구현하기    
[신경망 학습의 4단계]
1단계 : 데이터 중 일부를 무작위로 가져와 미니배치를 만든다.
2단계 : 손실함수를 최소로 하기 위하여 기울기를 구한다.
3단계 : 기울기에 따라 가중치를 조금씩 갱신한다.
4단계 : 1~3단계 반복

=>확률적 경사 하강법(SDG)

In [1]:
#4.5.1 2층 신경망 클래스 구현
import sys, os
import numpy as np
sys.path.append(os.path.abspath('../../../'))
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:  #클래스 생성
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01): #입력층의 뉴런 수 / 은닉층의 뉴런 수 / 출력층의 뉴런 수 / 
        #1/100로 나누어 정규화st   Q)<- 저걸 왜 해주는거지? 일단은 0으로 초기화했음 이것이 나중에 신경망 학습의 성공을 좌지우지 할수도,,,
        #B.가중치 초기화
        self.params = {}    #신경망 매개변수 보관하는 딕셔너리 변수(인스턴스 변수) ['W1']=1번층의 가중치 params['b1]=1번층의 편향
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)    #input_size와 hidden_size로 하는 이차원 배열에 임의의 값을 넣어준다. =>가중치 초기화                                                                                                                                                                                                                                                                                                                                                                            
        self.params['b1'] = np.zeros(hidden_size)   
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
                                              
    def predict(self, x):  #예측(추론), x는 이미지 데이터
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        return y

    #x : 입력데이터, t : 정답 레이블
    def loss(self, x, t):   #손실함수, x는 이미지 데이터 / t는 정답 레이블
        y = self.predict(x)
        
        return cross_entropy_error(y,t) #교차 엔트로피 오차를 이용하였음 
    
    def accuracy(self, x, t):   #정확도구하기      
         
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    
    #x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W:self.loss(x,t)
        
        grads = {}  #기울기 보관 딕셔너리 변수 
        grads['W1'] = numerical_gradient(loss_W, self.params['W1']) #grads['W1']=1번층의 가중치의 기울기 => 가중치로 미분했다.
        grads['b1'] = numerical_gradient(loss_W, self.params['b1']) #grads['b1']=1번층의 편향의 기울기 => 편향으로 미분했다.
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

+
Q)class의 self의 의미는 무엇인가?   
A)말그대로 자기 자신이라는 뜻이다. 실제로 class를 선언 후 사용하게 될 때 self부분에 선언한 변수를 써주면 된다.
E)def class:

In [4]:
#params => 가중치 매개변수를 저장해둔 딕셔너리이다.
    # def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    #     #B.가중치 초기화
    #     self.params = {}    #신경망 매개변수 보관하는 딕셔너리 변수(인스턴스 변수)
    #     self.params['W1'] = weight_init_std * \
    #                         np.random.randn(input_size, hidden_size)    #랜덤으로 input사이즈에서 hidden사이즈 만큼 뽑아온다.                                                                                                                                                                                                                                                                                                                                                                               
    #     self.params['b1'] = np.zeros(hidden_size)
    #     self.params['W2'] = weight_init_std * \
    #                         np.random.randn(hidden_size, output_size)
    #     self.params['b2'] = np.zeros(output_size)
                                              


net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


In [ ]:
#예측처리 실행하기
x = np.random.rand(100, 784)    #더미 입력 데이터(100장 분량), =>의미를 그대로 받아들이면 100,784만큼의 배열을 만들어준다.
y = net.predict(x)  

In [6]:
#grads => 기울이 저장 딕셔너리 변수
x = np.random.rand(100, 784)    #B.더미 입력 데이터 (100장 분량)
t = np.random.rand(100, 10)     #B.더미 정답 레이블 (100장 분량)

grads = net.numerical_gradient(x, t)    #기울기 계산 

print(grads['W1'].shape)
print(grads['b1'].shape)
print(grads['W2'].shape)
print(grads['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


In [10]:
#4.5.2 미니배치 학습 구현
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize = True, one_hot_label=True)    #(이미지 데이터, 정답 레이블 데이터)
                                                        #정규화(255로 나누기) 해주고 원핫 인코딩 해줌
train_loss_list = []    #손실함수 리스트 ??

#하이퍼파라미터 => 컴퓨터가 알아서 학습하도록 인간이 정해주는 값
iters_num = 10000   #반복 횟수
train_size = x_train.shape[0]   #이미지 데이터가 몇장인지
batch_size = 100    #미니배치 크기
learning_rate = 0.1 #학습률
network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

for i in range(iters_num):
    #미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)       #choice train_size(0~9999)숫자에서 batch_size(100개)개만큼의 임의의 숫자를 뽑아온다.
    x_batch = x_train[batch_mask]                               #batch_mask => 걸려서 본다는 느낌 st
    t_batch = t_train[batch_mask]
    
    #기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    #grad = network.gradient(x_batch, t_batch)  #성능 개선판 => 나중에 알려준다!
    
    #매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]    #원래 매개변수 값에서 학습률*기울기를 빼준다. 
        
    #학습 경과 기록
    loss = network.loss(x_batch, t_batch)   #오차 값을 기록해준다.
    train_loss_list.append(loss)            #오차 값을 리스트로 추가해주어서 학습 과정을 볼 수 있다.
    

ModuleNotFoundError: No module named 'two_layer_net'

[[질문]]      
책에서       
배치는 일일히 모든 데이터를 관찰해야한다.        
따라서 데이터가 너무 클 경우 미치배치를 활용하면    
다 보지도 않고 학습할 수 있음     
근데 이 코드에서는 미니배치를 이용하여 반복하고 있음     
이유가 뭘까?

In [8]:
#4.5.3 시험 데이터로 평가하기

위의 코드로 학습경과 기록 그래프를 살펴보면      
잘 학습하고 있음을 알 수 있음      
(왜냐 반복 횟수가 늘어날수록 손실함수 값이 줄어들으니)    
하지만 오버피팅이 되었는지는 확인 X 따라서 시험 데이터로 평가해보자 

***에폭***:   
단위이다. 훈련 데이터를 모두 소진했을 때의 횟수에 해당.   
ex) 10,000개의 데이터를 100개의 미니배치로 학습 시 100회가 1에폭이 된다.

In [15]:
#4.5.2 미니배치 학습 구현
import numpy as np
from dataset.mnist import load_mnist


(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize = True, one_hot_label=True)    #(이미지 데이터, 정답 레이블 데이터)
                                                        #정규화(255로 나누기) 해주고 원핫 인코딩 해줌

#하이퍼파라미터 => 컴퓨터가 알아서 학습하도록 인간이 정해주는 값
iters_num = 10000   #반복 횟수
train_size = x_train.shape[0]   #이미지 데이터가 몇장인지
batch_size = 100    #미니배치 크기
learning_rate = 0.1 #학습률

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

train_loss_list = []    #밑에 손실함수 값을 리스트로 만들것이기 때문에 선언 
train_acc_list = []     #정확도 측정 리스트
test_acc_list = []      #??

iter_per_epoch = max(train_size / batch_size, 1)   #1이 의미하는것이 뭘까? 둘중에 큰값을 고르라고 한것 batch_size가 train_size보다 밑에 큰 경우 나눗셈을 하기 위함

for i in range(iters_num):
    #미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)       #choice train_size(0~9999)숫자에서 batch_size(100개)개만큼의 임의의 숫자를 뽑아온다.
    x_batch = x_train[batch_mask]                               #batch_mask => 걸려서 본다는 느낌 st
    t_batch = t_train[batch_mask]
    
    #기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    #grad = network.gradient(x_batch, t_batch)  #성능 개선판 => 나중에 알려준다!
    
    #매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]    #원래 매개변수 값에서 학습률*기울기를 빼준다. 
        
    #학습 경과 기록
    loss = network.loss(x_batch, t_batch)   #오차 값을 기록해준다.
    train_loss_list.append(loss)            #오차 값을 리스트로 추가해주어서 학습 과정을 볼 수 있다.
    
    # 1에폭당 정확도 계산 (오버피팅 되었나 안되었나 확인)
    if i % iter_per_epoch == 0:     #왜 1에폭당일까? => i를 1에폭당 나눈것이면 데이터 전체를 1회 보게 되는 것이므로
        train_acc = network.accuracy(x_train, t_train)  #학습한 데이터에 대한 정확도
        test_acc = network.accuracy(x_test, t_test)     #새로운 데이터에 대한 정확도
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | "+ str(train_acc)+","+str(test_acc))   

train acc, test acc | 0.10441666666666667,0.1028


KeyboardInterrupt: 